- DateL Jun 29, 2024

In [1]:
# https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md

In [22]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

print(f"- courses: {len(documents_raw)}")

print(f"\n- course and corresponding documents: \n{[(d['course'], len(d['documents'])) for d in documents_raw]}")

print(f"\n- Sample document:\n{documents_raw[0]['documents'][0]}")


- courses: 3

- course and corresponding documents: 
[('data-engineering-zoomcamp', 435), ('machine-learning-zoomcamp', 375), ('mlops-zoomcamp', 138)]

- Sample document:
{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?'}


In [23]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [26]:
len(documents)

948

## Search using ElasticSearch

(venv_llmzc) manas:(llm-zoomcamp):~$ docker run -it \                                                                                           24-06-23 @ 17:06
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [28]:
from elasticsearch import Elasticsearch

In [29]:
!curl http://localhost:9200

{
  "name" : "9f8179841cc6",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "T1KZKZcrSW2PkNaWUDeWcA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [30]:
es_client = Elasticsearch('http://localhost:9200') 

In [31]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [32]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-hw"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-hw'})

In [33]:
from tqdm.auto import tqdm

/Users/manaschaity/Documents/python_venvs/venv_llmzc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:01<00:00, 483.83it/s]


## Q3. Searching

In [37]:
# query = 'I just disovered the course. Can I still join it?'
query = "How do I execute a command in a running docker container?"

In [40]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
print(response)
result_docs = []

{'took': 17, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions-hw', '_id': 't-LmZZABWUz7CXfHUiK0', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions-hw', '_id': 'xeLmZZABWUz7CXfHTiC5', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container in

## Q4. Filtering

In [41]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

import pprint
pprint.pp(response)


ObjectApiResponse({'took': 23, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions-hw', '_id': 't-LmZZABWUz7CXfHUiK0', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions-hw', '_id': '1uLmZZABWUz7CXfHUiLv', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it

In [45]:
print(response.body)

{'took': 23, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions-hw', '_id': 't-LmZZABWUz7CXfHUiK0', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions-hw', '_id': '1uLmZZABWUz7CXfHUiLv', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file 

## Q5. Building a prompt

In [47]:
result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [64]:
context_template = """
Q: {question}
A: {text}
""".strip()

complete_context = ''
for i, context_entry in enumerate(result_docs):
    qts = context_entry['question']
    txt = context_entry['text']
    
    hydrated_prompt = context_template.format(question=qts, text=txt).strip()

    if i == 0:
        complete_context = hydrated_prompt
    else:
        complete_context = complete_context + "\n\n" + hydrated_prompt
    # print(hydrated_prompt)

print(complete_context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [65]:
# query = "How do I execute a command in a running docker container?"
query

'How do I execute a command in a running docker container?'

In [66]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [67]:
full_prompt = prompt_template.format(question=query, context=complete_context).strip()
print(full_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [68]:
len(full_prompt)

1462

## Q6. Tokens

In [72]:
# !pip install tiktoken 

In [74]:
import tiktoken 
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [76]:
encoded_full_prompt = encoding.encode(full_prompt)
print(encoded_full_prompt)

[2675, 2351, 264, 3388, 12917, 18328, 13, 22559, 279, 91022, 3196, 389, 279, 88436, 505, 279, 32072, 4729, 627, 10464, 1193, 279, 13363, 505, 279, 88436, 994, 36864, 279, 91022, 382, 53528, 25, 2650, 656, 358, 9203, 264, 3290, 304, 264, 4401, 27686, 5593, 1980, 99465, 512, 48, 25, 2650, 656, 358, 7542, 264, 27686, 5593, 5380, 32, 25, 24083, 279, 5593, 2217, 304, 21416, 3941, 323, 68971, 279, 4441, 2837, 11, 779, 430, 433, 8638, 264, 28121, 3290, 627, 29748, 1629, 482, 275, 1198, 4177, 2837, 28121, 366, 1843, 397, 2746, 279, 5593, 374, 2736, 4401, 11, 9203, 264, 3290, 304, 279, 3230, 5593, 512, 29748, 4831, 320, 3990, 279, 5593, 13193, 340, 29748, 3969, 482, 275, 366, 3670, 13193, 29, 28121, 198, 3269, 8362, 437, 386, 51015, 696, 48, 25, 2650, 656, 358, 3048, 3626, 505, 856, 2254, 5780, 311, 27686, 5593, 5380, 32, 25, 1472, 649, 3048, 3626, 505, 701, 2254, 5780, 1139, 264, 41649, 5593, 1701, 279, 27686, 12773, 3290, 13, 5810, 596, 1268, 311, 656, 433, 512, 1271, 3048, 264, 1052, 477, 63

In [77]:
len(encoded_full_prompt)

325

In [78]:
encoding_1 = tiktoken.encoding_for_model("gpt-4o")

In [80]:
encoded_1_full_prompt = encoding_1.encode(full_prompt)
len(encoded_1_full_prompt)

322